In [107]:
import pandas as pd
import numpy as np
import seaborn as sns
sns.set()
import matplotlib.pyplot as plt
%matplotlib inline

In [108]:
train = pd.read_csv('train_2kmZucJ.csv')
train.head()

,id,label,tweet
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1...
1,2,0,Finally a transparant silicon case ^^ Thanks t...
2,3,0,We love this! Would you go? #talk #makememorie...
3,4,0,I'm wired I know I'm George I was made that wa...
4,5,1,What amazing service! Apple won't even talk to...


In [109]:
test = pd.read_csv('test_oJQbWVk.csv')
test.head()

,id,tweet
0,7921,I hate the new #iphone upgrade. Won't let me d...
1,7922,currently shitting my fucking pants. #apple #i...
2,7923,"I'd like to puts some CD-ROMS on my iPad, is t..."
3,7924,My ipod is officially dead. I lost all my pict...
4,7925,Been fighting iTunes all night! I only want th...


In [110]:
for index, tweet in enumerate(train["tweet"][10:15]):
    print(index+1,".",tweet)

1 . hey #apple when you make a new ipod dont make it a new color or .2inches thinner make it not crash every five fuckin minite.
2 . Ha! Not heavy machinery but it does what I need it to. @Apple really dropped the ball with that design. #drinkyourhaterade
3 . Contemplating giving in to the iPhone bandwagon simply because #Cellcom has no new Androids #depressing #IDontWantTo
4 . I just made another crazy purchase lol my theory is 'Work hard, Play Hard' Lol #iPad #Apple #Shopping http://instagr.am/p/NUyFHvyE7I/
5 . @shaqlockholmes @sam_louise1991 the battery is so painful! I charge it overnight and by lunchtime battery is dead! #HateOrange.


In [111]:
import re

#Clean text from noise
def clean_text(text):
    #Filter to allow only alphabets
    text = re.sub(r'[^a-zA-Z\']', ' ', text)
    
    #Remove Unicode characters
    text = re.sub(r'[^\x00-\x7F]+', '', text)
    
    #Convert to lowercase to maintain consistency
    text = text.lower()
       
    return text

In [112]:
train['clean_text'] = train.tweet.apply(lambda x: clean_text(x))

In [113]:
#Exhaustive list of stopwords in the english language. We want to focus less on these so at some point will have to filter
STOP_WORDS = ['a', 'about', 'above', 'after', 'again', 'against', 'all', 'also', 'am', 'an', 'and',
              'any', 'are', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below',
              'between', 'both', 'but', 'by', 'can', "can't", 'cannot', 'com', 'could', "couldn't", 'did',
              "didn't", 'do', 'does', "doesn't", 'doing', "don't", 'down', 'during', 'each', 'else', 'ever',
              'few', 'for', 'from', 'further', 'get', 'had', "hadn't", 'has', "hasn't", 'have', "haven't", 'having',
              'he', "he'd", "he'll", "he's", 'her', 'here', "here's", 'hers', 'herself', 'him', 'himself', 'his', 'how',
              "how's", 'however', 'http', 'i', "i'd", "i'll", "i'm", "i've", 'if', 'in', 'into', 'is', "isn't", 'it',
              "it's", 'its', 'itself', 'just', 'k', "let's", 'like', 'me', 'more', 'most', "mustn't", 'my', 'myself',
              'no', 'nor', 'not', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'otherwise', 'ought', 'our', 'ours',
              'ourselves', 'out', 'over', 'own', 'r', 'same', 'shall', "shan't", 'she', "she'd", "she'll", "she's",
              'should', "shouldn't", 'since', 'so', 'some', 'such', 'than', 'that', "that's", 'the', 'their', 'theirs',
              'them', 'themselves', 'then', 'there', "there's", 'these', 'they', "they'd", "they'll", "they're",
              "they've", 'this', 'those', 'through', 'to', 'too', 'under', 'until', 'up', 'very', 'was', "wasn't",
              'we', "we'd", "we'll", "we're", "we've", 'were', "weren't", 'what', "what's", 'when', "when's", 'where',
              "where's", 'which', 'while', 'who', "who's", 'whom', 'why', "why's", 'with', "won't", 'would', "wouldn't",
              'www', 'you', "you'd", "you'll", "you're", "you've", 'your', 'yours', 'yourself', 'yourselves']

#Generate word frequency
def gen_freq(text):
    #Will store the list of words
    word_list = []

    #Loop over all the tweets and extract words into word_list
    for tw_words in text.split():
        word_list.extend(tw_words)

    #Create word frequencies using word_list
    word_freq = pd.Series(word_list).value_counts()
    
    #Drop the stopwords during the frequency calculation
    word_freq = word_freq.drop(STOP_WORDS, errors='ignore')
    
    return word_freq

#Check whether a negation term is present in the text
def any_neg(words):
    for word in words:
        if word in ['n', 'no', 'non', 'not'] or re.search(r"\wn't", word):
            return 1
    else:
        return 0

#Check whether one of the 100 rare words is present in the text
def any_rare(words, rare_100):
    for word in words:
        if word in rare_100:
            return 1
    else:
        return 0

#Check whether prompt words are present
def is_question(words):
    for word in words:
        if word in ['when', 'what', 'how', 'why', 'who']:
            return 1
    else:
        return 0

In [114]:
word_freq = gen_freq(train.clean_text.str)
#100 most rare words in the dataset
rare_100 = word_freq[-100:]
#Number of words in a tweet
train['word_count'] = train.clean_text.str.split().apply(lambda x: len(x))
#Negation present or not
train['any_neg'] = train.clean_text.str.split().apply(lambda x: any_neg(x))
#Prompt present or not
train['is_question'] = train.clean_text.str.split().apply(lambda x: is_question(x))
#Any of the most 100 rare words present or not
train['any_rare'] = train.clean_text.str.split().apply(lambda x: any_rare(x, rare_100))
#Character count of the tweet
train['char_count'] = train.clean_text.apply(lambda x: len(x))

In [115]:
gen_freq(train.clean_text.str)[:10]

iphone       4313
apple        2974
p            2784
instagram    2187
samsung      1441
twitter      1262
new          1160
https        1014
phone         998
sony          855
dtype: int64

In [116]:
train.head()

,id,label,tweet,clean_text,word_count,any_neg,is_question,any_rare,char_count
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1...,fingerprint pregnancy test https goo gl h ...,17,0,0,0,128
1,2,0,Finally a transparant silicon case ^^ Thanks t...,finally a transparant silicon case thanks t...,21,0,0,0,131
2,3,0,We love this! Would you go? #talk #makememorie...,we love this would you go talk makememorie...,19,1,0,0,123
3,4,0,I'm wired I know I'm George I was made that wa...,i'm wired i know i'm george i was made that wa...,22,0,0,0,112
4,5,1,What amazing service! Apple won't even talk to...,what amazing service apple won't even talk to...,22,1,1,0,124


In [117]:
from sklearn.model_selection import train_test_split

X = train[['word_count', 'any_neg', 'any_rare', 'char_count', 'is_question']]
y = train.label

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=27)

In [118]:
from sklearn.naive_bayes import GaussianNB

#Initialize GaussianNB classifier
model = GaussianNB()
#Fit the model on the train dataset
model = model.fit(X_train, y_train)
#Make predictions on the test dataset
pred = model.predict(X_test)

In [119]:
from sklearn.metrics import accuracy_score

print("Accuracy:", accuracy_score(y_test, pred)*100, "%")

Accuracy: 76.89393939393939 %


In [120]:
for index, tweet in enumerate(test["tweet"][10:15]):
    print(index+1,".",tweet)

1 . Con mi buddy#edgar #buddy #friend #viviendo #taller #friendly #hug #sony #smile #sonrisa http://instagram.com/p/fasVZruzQI/
2 . #MyMomIsGreatBecause she always care about me! And do anything for me #FOLLOW #MothersDay #MustFollow mothersday #iPhone #tweet
3 . #kiss,,#romance,#hot,#movie,#iphone,#sex,#porn Lustful Mature Woman Is Getting Nailed Intensively In A Mission http://bit.ly/SPBTUR
4 . Holding to hope when a loved one is… http://dlvr.it/LQjnNj #news #photography #fashion #health #fail #tech #ipad #iphone #funny #lol
5 . New phone case :) #Me #Selfie #Guy #Dude #Smiles #Teeth #Cute #Samsung #GalaxyS3 #British #Flag #Vint http://instagr.am/p/Q_dqXztofj/


In [121]:
import re

#Clean text from noise
def clean_text(text):
    #Filter to allow only alphabets
    text = re.sub(r'[^a-zA-Z\']', ' ', text)
    
    #Remove Unicode characters
    text = re.sub(r'[^\x00-\x7F]+', '', text)
    
    #Convert to lowercase to maintain consistency
    text = text.lower()
       
    return text

In [122]:
test['clean_text'] = test.tweet.apply(lambda x: clean_text(x))

In [123]:
#Exhaustive list of stopwords in the english language. We want to focus less on these so at some point will have to filter
STOP_WORDS = ['a', 'about', 'above', 'after', 'again', 'against', 'all', 'also', 'am', 'an', 'and',
              'any', 'are', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below',
              'between', 'both', 'but', 'by', 'can', "can't", 'cannot', 'com', 'could', "couldn't", 'did',
              "didn't", 'do', 'does', "doesn't", 'doing', "don't", 'down', 'during', 'each', 'else', 'ever',
              'few', 'for', 'from', 'further', 'get', 'had', "hadn't", 'has', "hasn't", 'have', "haven't", 'having',
              'he', "he'd", "he'll", "he's", 'her', 'here', "here's", 'hers', 'herself', 'him', 'himself', 'his', 'how',
              "how's", 'however', 'http', 'i', "i'd", "i'll", "i'm", "i've", 'if', 'in', 'into', 'is', "isn't", 'it',
              "it's", 'its', 'itself', 'just', 'k', "let's", 'like', 'me', 'more', 'most', "mustn't", 'my', 'myself',
              'no', 'nor', 'not', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'otherwise', 'ought', 'our', 'ours',
              'ourselves', 'out', 'over', 'own', 'r', 'same', 'shall', "shan't", 'she', "she'd", "she'll", "she's",
              'should', "shouldn't", 'since', 'so', 'some', 'such', 'than', 'that', "that's", 'the', 'their', 'theirs',
              'them', 'themselves', 'then', 'there', "there's", 'these', 'they', "they'd", "they'll", "they're",
              "they've", 'this', 'those', 'through', 'to', 'too', 'under', 'until', 'up', 'very', 'was', "wasn't",
              'we', "we'd", "we'll", "we're", "we've", 'were', "weren't", 'what', "what's", 'when', "when's", 'where',
              "where's", 'which', 'while', 'who', "who's", 'whom', 'why', "why's", 'with', "won't", 'would', "wouldn't",
              'www', 'you', "you'd", "you'll", "you're", "you've", 'your', 'yours', 'yourself', 'yourselves']

#Generate word frequency
def gen_freq(text):
    #Will store the list of words
    word_list = []

    #Loop over all the tweets and extract words into word_list
    for tw_words in text.split():
        word_list.extend(tw_words)

    #Create word frequencies using word_list
    word_freq = pd.Series(word_list).value_counts()
    
    #Drop the stopwords during the frequency calculation
    word_freq = word_freq.drop(STOP_WORDS, errors='ignore')
    
    return word_freq

#Check whether a negation term is present in the text
def any_neg(words):
    for word in words:
        if word in ['n', 'no', 'non', 'not'] or re.search(r"\wn't", word):
            return 1
    else:
        return 0

#Check whether one of the 100 rare words is present in the text
def any_rare(words, rare_100):
    for word in words:
        if word in rare_100:
            return 1
    else:
        return 0

#Check whether prompt words are present
def is_question(words):
    for word in words:
        if word in ['when', 'what', 'how', 'why', 'who']:
            return 1
    else:
        return 0

In [124]:
word_freq = gen_freq(test.clean_text.str)
#100 most rare words in the dataset
rare_100 = word_freq[-100:]
#Number of words in a tweet
test['word_count'] = test.clean_text.str.split().apply(lambda x: len(x))
#Negation present or not
test['any_neg'] = test.clean_text.str.split().apply(lambda x: any_neg(x))
#Prompt present or not
test['is_question'] = test.clean_text.str.split().apply(lambda x: is_question(x))
#Any of the most 100 rare words present or not
test['any_rare'] = test.clean_text.str.split().apply(lambda x: any_rare(x, rare_100))
#Character count of the tweet
test['char_count'] = test.clean_text.apply(lambda x: len(x))

In [125]:
#Top 10 common words are
gen_freq(test.clean_text.str)[:10]

iphone       1007
apple         769
p             701
instagram     557
samsung       359
new           293
twitter       278
phone         239
https         236
sony          206
dtype: int64

In [126]:
test.head()

,id,tweet,clean_text,word_count,any_neg,is_question,any_rare,char_count
0,7921,I hate the new #iphone upgrade. Won't let me d...,i hate the new iphone upgrade won't let me d...,14,1,0,0,77
1,7922,currently shitting my fucking pants. #apple #i...,currently shitting my fucking pants apple i...,16,0,0,0,115
2,7923,"I'd like to puts some CD-ROMS on my iPad, is t...",i'd like to puts some cd roms on my ipad is t...,21,1,0,0,104
3,7924,My ipod is officially dead. I lost all my pict...,my ipod is officially dead i lost all my pict...,24,0,0,0,129
4,7925,Been fighting iTunes all night! I only want th...,been fighting itunes all night i only want th...,13,0,0,0,70


In [127]:
test = test.drop('tweet', axis=1)

In [128]:
test = test.drop('clean_text', axis=1)

In [129]:
submission['id']=test['id']

In [130]:
test = test.drop('id', axis=1)

In [131]:
pred_test = model.predict(test)

In [132]:
submission=pd.read_csv("sample_submission_LnhVWA4.csv")

In [133]:
submission['label']=pred_test 

In [134]:
pd.DataFrame(submission, columns=['id','label']).to_csv('SentimentsLabel.csv')